In [12]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import cv2 as cv 

import torch 
from torch import nn 
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models 

## Device Config


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyper Parameters

In [14]:
num_epochs = 2 
batch_size = 4
learning_rate = 0.001


transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))])

## CIFAR1

### Download Dataset

In [15]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', 
                                           train=True,
                                           transform= transform,
                                           download=True)


test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False,
                                            download=True,
                                            transform= transform)

Files already downloaded and verified
Files already downloaded and verified


### Loading Train and Test dataset 

In [16]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [17]:
# Defining Classes

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 
           'frog', 'horse', 'ship', 'truck')


### Build Model

In [18]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400 # flatten the images
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x


model = ConvNet().to(device)

### Evaluation Methods

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Training Loop

In [20]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/2], Step [2000/12500], Loss: 2.2704
Epoch [1/2], Step [4000/12500], Loss: 2.2823
Epoch [1/2], Step [6000/12500], Loss: 2.2781
Epoch [1/2], Step [8000/12500], Loss: 2.1086
Epoch [1/2], Step [10000/12500], Loss: 2.1319
Epoch [1/2], Step [12000/12500], Loss: 1.8860
Epoch [2/2], Step [2000/12500], Loss: 2.0209
Epoch [2/2], Step [4000/12500], Loss: 2.0738
Epoch [2/2], Step [6000/12500], Loss: 1.8505
Epoch [2/2], Step [8000/12500], Loss: 1.1538
Epoch [2/2], Step [10000/12500], Loss: 1.8532
Epoch [2/2], Step [12000/12500], Loss: 1.2287


In [21]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 35.02 %
Accuracy of plane: 43.8 %
Accuracy of car: 22.4 %
Accuracy of bird: 11.1 %
Accuracy of cat: 11.5 %
Accuracy of deer: 26.8 %
Accuracy of dog: 36.7 %
Accuracy of frog: 63.6 %
Accuracy of horse: 37.3 %
Accuracy of ship: 26.8 %
Accuracy of truck: 70.2 %
